In [13]:
# 1) Load env vars and create engine
import os
from dotenv import load_dotenv
import pandas as pd
from sqlalchemy import create_engine

load_dotenv()

pg_user     = os.getenv("PG_USER")
pg_password = os.getenv("PG_PASSWORD")
pg_host     = os.getenv("PG_HOST")
pg_db       = os.getenv("PG_DB")

conn_str = f"postgresql+psycopg2://{pg_user}:{pg_password}@{pg_host}/{pg_db}"
engine   = create_engine(conn_str)

# 2) Show all rows in results
pd.set_option("display.max_rows", None)

## Web-Scrape Descriptive Analytics
**Business question:** Which five channels generate the most total reactions (likes + comments), and what is each channel’s top video by reaction count?

In [14]:
api_desc_sql = """
WITH channel_totals AS (
  -- total reactions per channel
  SELECT
    channel_name,
    SUM(like_count + comment_count) AS total_reactions
  FROM raw.yt_video
  GROUP BY channel_name
),
video_reacts AS (
  -- per-video reaction counts
  SELECT
    v.video_id,
    v.video_name,
    v.channel_name,
    (v.like_count + v.comment_count) AS video_reactions
  FROM raw.yt_video AS v
),
ranked_videos AS (
  -- join channel totals to each video and rank within channel
  SELECT
    vr.video_id,
    vr.video_name,
    vr.channel_name,
    vr.video_reactions,
    ct.total_reactions,
    ROW_NUMBER() OVER (
      PARTITION BY vr.channel_name
      ORDER BY vr.video_reactions DESC
    ) AS video_rank
  FROM video_reacts vr
  JOIN channel_totals ct
    ON vr.channel_name = ct.channel_name
)
SELECT
  channel_name,
  total_reactions,
  video_id        AS top_video_id,
  video_name      AS top_video_name,
  video_reactions AS top_video_reactions
FROM ranked_videos
WHERE video_rank = 1
ORDER BY total_reactions DESC
LIMIT 5;
"""
df_api_desc = pd.read_sql(api_desc_sql, engine)
df_api_desc


,channel_name,total_reactions,top_video_id,top_video_name,top_video_reactions
0,HYBE LABELS,972000.0,R2-yomhYAj4,"KATSEYE (캣츠아이) ""Gnarly"" Official MV",492100
1,Maroon5VEVO,672600.0,vaGf8fmtBr4,Maroon 5 - Priceless ft. LISA (Official Video),672600
2,Anuel AA,348100.0,CVt4nCsJnzg,Anuel AA - BUGATTI (Video Oficial),348100
3,Young Thug,314500.0,W4aE8of2znM,Young Thug - Money On Money (feat. Future) [Of...,314500
4,YoungBoy Never Broke Again,307600.0,F7dJY0jkpyU,YoungBoy Never Broke Again - Where I Been / Sh...,307600


**Insight:**

- HYBE LABELS leads by a wide margin with 972 000 total reactions, driven chiefly by “Gnarly” (492 100 reactions).

- Maroon5VEVO follows at 672 600, entirely fueled by “Maroon 5 – Priceless ft. LISA.”

- The top five channels together account for over 2.6 million reactions, but individual channels vary: some (Maroon5VEVO) rely on a single hit video, while others (HYBE LABELS) see more distributed engagement across multiple uploads.

**Recommendation:**

- Replicate top‐video mechanics: Analyze thumbnails, release timing, and CTAs from “Gnarly” and “Priceless” to apply those elements broadly.

- Diversify content for single‐hit channels: schedule follow-up videos and spin-off clips to spread reaction load rather than depending on one blockbuster.

**Prediction:**

- Implementing these tactics—mimicking top thumbnails and ramping up follow-up content—should yield a 15–20% lift in total reactions across your top five channels in the next month.

## Web-Scrape Diagnostic Analytics
**Business question:** Which channels have the highest average like-to-comment ratio?

In [15]:
api_desc_sql = """
WITH
  video_ratios AS (
    -- compute like/comment ratio per video
    SELECT
      channel_name,
      CASE 
        WHEN comment_count = 0 THEN NULL
        ELSE like_count::numeric / comment_count 
      END AS like_comment_ratio
    FROM raw.yt_video
  ),
  channel_ratios AS (
    -- average that ratio per channel
    SELECT
      channel_name,
      AVG(like_comment_ratio) AS avg_like_comment_ratio
    FROM video_ratios
    GROUP BY channel_name
  ),
  channel_list AS (
    -- list of all channels (for a JOIN example)
    SELECT DISTINCT channel_name
    FROM raw.yt_video
  ),
  ranked_channels AS (
    -- rank channels by their avg ratio
    SELECT
      cr.channel_name,
      cr.avg_like_comment_ratio,
      RANK() OVER (
        ORDER BY cr.avg_like_comment_ratio DESC
      ) AS ratio_rank
    FROM channel_ratios cr
    JOIN channel_list cl
      ON cr.channel_name = cl.channel_name
  )
SELECT
  channel_name,
  ROUND(avg_like_comment_ratio, 4) AS avg_like_comment_ratio,
  ratio_rank
FROM ranked_channels
WHERE ratio_rank <= 5
ORDER BY ratio_rank;
"""
df_api_desc = pd.read_sql(api_desc_sql, engine)
df_api_desc

,channel_name,avg_like_comment_ratio,ratio_rank
0,Plan B - Topic,113.7405,1
1,Fuerza Regida - Topic,52.0832,2
2,Kyle Richh,32.2727,3
3,BIG30VEVO,25.7778,4
4,Tee Grizzley,23.8421,5


**Insight:** 

- Plan B – Topic sits at an exceptional 113.74 likes per comment, more than double the runner-up, Fuerza Regida – Topic (52.08).

- Channels like BIG30VEVO and Tee Grizzley show more balanced ratios (~25 – 23), indicating relatively stronger comment activity per view.

**Recommendation:**

- Drive deeper engagement on high‐ratio channels by embedding explicit comment prompts (e.g. “Tell us your favorite verse below!”).

- Host Q&A or polls on these channels to convert passive likes into active discussion, building Viewer Community and feedback loops.

**Prediction:**

- By integrating targeted CTAs and interactive elements, comment volumes on high-ratio channels should increase by 50%, bringing like-to-comment ratios closer to 40–50, a more balanced engagement profile.